In [1]:
import pandas as pd
import numpy as np

from src.pingers import ping_ads_insights_all_platforms, ping_shops
from src.database.session import db
from src.s3 import read_csv_from_s3
from src.models import *
from src.schemas import *
from src.crud import ga_daily_performance, crud_fb_daily_performance

2023-02-09 17:35:19.299 | DEBUG    | src.database.session:<module>:19 - base: prod.cw6tznt1p9hq.us-east-1.rds.amazonaws.com


base I am using: prod.cw6tznt1p9hq.us-east-1.rds.amazonaws.com


In [5]:
row = StreamlitUser(username='lebeg', hashed_password='lebeg123.', is_superuser=True)
db.add(row)

TypeError: 'username' is an invalid keyword argument for StreamlitUser

In [33]:
query = crud_fb_daily_performance.query_budget_split_by_target(db=db, shop_id=[16038, 2], start_date='2022-01-01')
str(query)

'SELECT facebook_daily_performance.shop_id AS facebook_daily_performance_shop_id, facebook_adset.target AS facebook_adset_target, sum(facebook_daily_performance.spend / anon_1.rate_from_usd) AS spend \nFROM facebook_daily_performance JOIN facebook_adset ON facebook_daily_performance.shop_id = facebook_adset.shop_id AND facebook_daily_performance.account_id = facebook_adset.account_id AND facebook_daily_performance.adset_id = facebook_adset.adset_id JOIN facebook_ad_account ON facebook_daily_performance.account_id = facebook_ad_account.facebook_id AND facebook_daily_performance.shop_id = facebook_ad_account.shop_id JOIN (SELECT DISTINCT ON (currency_exchange_rate.code) currency_exchange_rate.code AS code, currency_exchange_rate.rate_from_usd AS rate_from_usd \nFROM currency_exchange_rate ORDER BY currency_exchange_rate.code, currency_exchange_rate.date DESC) AS anon_1 ON facebook_ad_account.currency = anon_1.code \nWHERE facebook_daily_performance.shop_id IN (__[POSTCOMPILE_shop_id_1]) 

In [34]:
df = pd.read_sql(query.statement, db.bind)
df

,shop_id,target,spend
0,2,Target.acquisition,5159.64
1,2,Target.remarketing,221.28
2,16038,Target.acquisition,1690709.21
3,16038,Target.remarketing,167237.34


In [2]:
shops = ping_shops(db=db)

2023-02-09 14:46:01.495 | DEBUG    | src.pingers.ping_shops:ping_shops:23 - Here I am!


ping_shops: 0:00:19.410317


In [5]:
shop_id_query = db.query(FacebookDailyPerformance.shop_id).filter(FacebookDailyPerformance.date_start >= '2022-01-01').distinct()
shop_ids = pd.read_sql(shop_id_query.statement, db.bind)

In [6]:
shop_ids = shop_ids['shop_id'].tolist()
chunck_size = 3
for i, shop_ids_chunck in enumerate(np.array_split(shop_ids, len(shop_ids) // chunck_size)):
    print(shop_ids_chunck.tolist())
    query = crud_fb_daily_performance.query_budget_split_by_target(db=db, shop_id=shop_ids_chunck.tolist(), start_date='2022-01-01')
    print(query)
    new_df = pd.read_sql(query.statement, db.bind)
    if i == 0:
        df = new_df
    else:
        df = pd.concat([df, new_df], axis=0)
    if i == 2:
        break

[2, 96, 15873, 16038]
SELECT facebook_daily_performance.shop_id AS facebook_daily_performance_shop_id, facebook_adset.target AS facebook_adset_target, sum(facebook_daily_performance.spend / anon_1.rate_from_usd) AS spend 
FROM facebook_daily_performance JOIN facebook_adset ON facebook_daily_performance.shop_id = facebook_adset.shop_id AND facebook_daily_performance.account_id = facebook_adset.account_id AND facebook_daily_performance.adset_id = facebook_adset.adset_id JOIN facebook_ad_account ON facebook_daily_performance.account_id = facebook_ad_account.facebook_id AND facebook_daily_performance.shop_id = facebook_ad_account.shop_id JOIN (SELECT DISTINCT ON (currency_exchange_rate.code) currency_exchange_rate.code AS code, currency_exchange_rate.rate_from_usd AS rate_from_usd 
FROM currency_exchange_rate ORDER BY currency_exchange_rate.code, currency_exchange_rate.date DESC) AS anon_1 ON facebook_ad_account.currency = anon_1.code 
WHERE facebook_daily_performance.shop_id IN (__[POSTCO

[array([36833384, 46864466, 43739609, 50592831, 36370064, 53002252,
        39579188, 31971959, 43985342, 45844908, 38489951, 20347698,
        68534913, 12153503, 33391601, 36143108, 12109463, 43242730,
        41490637, 13597214, 48425409, 31064070, 15332300, 13781927,
        43352241,   256652,  1370565,  2114125, 17807133, 51022062,
        31839270, 38580607, 37140876, 44270769,  1407292, 41224141,
        50604700, 49063542, 45008008, 76522436, 50254205, 31904124,
        21313648, 13131861, 41174878, 50783576, 17045701, 40954863,
           15887, 40330230, 22926764,  9596262, 45038286,  8573833,
        44512322, 38092292, 45836491, 39270994, 17506226, 56579596,
        38833446, 74399413, 41363055, 38962439, 47140968,  8112778,
        44086229, 45045056, 46360217, 38894179, 18675218, 30704322,
        46317801, 41273036, 43954850, 12140410, 45764574,  9231199,
        25963515, 31623603, 40281047,    16006, 39329571, 27227973,
         9893977, 49074615, 41020843, 58264127, 

In [11]:
def f(df):
    print(df)
    df.spend = df.spend / df.spend.sum()
    return df

In [12]:
df.groupby('shop_id').transform(f)

0    Target.acquisition
1    Target.remarketing
Name: target, dtype: object


AttributeError: 'Series' object has no attribute 'spend'

In [ ]:
import pandas as pd
# crm = pd.read_csv('data/industries.csv')
df = pd.read_csv('data/ads_insights.csv')

In [ ]:
q = 4
pd.qcut(df.loc[df.facebook_spend.notna(), 'facebook_spend'], q=q, labels=list(range(q))).isna().sum()

In [ ]:
df.industry.count()

In [ ]:
import pandas as pd
from enum import Enum
from itertools import combinations
from src.utils.enum import get_enum_values
import numpy as np

class EPlatform(str, Enum):
    facebook = "facebook"
    tiktok = "tiktok"
    google = "google"

df = pd.read_csv('data/ads_insights.csv')

In [ ]:
"-".join(['a', 'b'])

In [ ]:
set(combinations(, r=2))

In [ ]:
def get_all_subsets(s: set | list):
    n = len(s)
    subsets = []
    for i in range(n+1):
        i_subsets = combinations(s, i)
        subsets.extend(i_subsets)
    return subsets

In [ ]:
platforms = get_enum_values(EPlatform)
platform_combinations = get_all_subsets(platforms)

In [ ]:
table = pd.DataFrame(columns=['combination', 'shops', 'budget_split'])

In [ ]:
def get_spend_ratio(s: pd.Series, platforms: list):
    ratio = [s[f"{platform}_spend"] / s.full_spend for platform in platforms]
    return ratio

In [ ]:
def get_data_by_platform_combination(df: pd.DataFrame, combination: list):
    filter = [True in range(len(df))]
    for platform in platforms:
        if platform in combination:
            filter = filter & (df[f'{platform}_spend'].notna())
        else:
            filter = filter & (df[f'{platform}_spend'].isna())
    filtered = df[filter].copy()
    filtered['full_spend'] = filtered.apply(lambda df: sum([df[f"{platform}_spend"] for platform in combination]), axis=1)
    platforms_without_the_last_one = combination[:len(combination)-1]
    filtered['spend_ratio'] = filtered.apply(lambda df: get_spend_ratio(df, platforms=platforms_without_the_last_one), axis=1)
    mean_ratio = np.mean(filtered['spend_ratio'].tolist(), axis=0)
    print(mean_ratio)
    result = {"combination": '_'.join(combination), "shops": len(filtered)/len(df), "budget_split": list(mean_ratio) + [1-mean_ratio.sum()]}
    return result

In [ ]:
for combination in platform_combinations:
    if not len(combination):
        continue
    row = get_data_by_platform_combination(df=df, combination=combination)
    table.loc[len(table), :] = row

In [ ]:
table

In [ ]:
result

In [ ]:
filtered

In [ ]:
filtered.spend_ratio

In [ ]:
filtered

In [ ]:
combination[:len(combination)-1]

In [ ]:
len(df[(df.facebook_spend.notna()) & (df.google_spend.isna()) & (df.tiktok_spend.notna())])

In [ ]:
df = ping_ads_insights_all_platforms(db=db, columns='spend')

In [ ]:
only_shop = df.groupby('shop_id').sum().reset_index()

In [ ]:
df

In [ ]:
FacebookAd.ad_id.__mapper__

In [ ]:
FacebookAd.__class__

In [ ]:
df = ping_shops(db=db)

In [ ]:
df.loc[df.shop_id == 16038, :]

In [ ]:
df.ad_id

In [ ]:
df.loc[df.ad_id == '10'].index[0]

In [ ]:
query = session.query(FacebookAd.ad_id)

In [ ]:
query[0]

In [ ]:
type(query)